# MDTraj NetCDF Reporter

Tested through OpenMM reporter interface.

See OpenMM issue: https://github.com/openmm/openmm/issues/4298

See MDTraj issue: https://github.com/mdtraj/mdtraj/issues/1831

Sample system taken from OpenMM Cookbook: https://openmm.github.io/openmm-cookbook/latest/notebooks/tutorials/protein_in_water.html

In [ ]:
!wget https://files.rcsb.org/download/1AKI.pdb

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

pdb = PDBFile("1AKI.pdb")

In [2]:
# Specify the forcefield
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

In [3]:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.deleteWater()
residues=modeller.addHydrogens(forcefield)

In [4]:
# forget about solvent for demonstration
# modeller.addSolvent(forcefield, padding=1.0*nanometer)

In [5]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=0.5*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

In [6]:
#print("Minimizing energy")
simulation.minimizeEnergy()

## Use `NetCDF` reporter

This simulation + reporter set is run in an environment with `netCDF4` installed.

In [7]:
from mdtraj.reporters import NetCDFReporter

In [8]:
reporter1 = NetCDFReporter('trajectory_netcdf4.nc', 1000)
reporter2 = NetCDFReporter('trajectory_netcdf4_100.nc', 10)
print(type(reporter1._traj_file._handle))

<class 'netCDF4._netCDF4.Dataset'>


In [9]:
simulation.reporters.append(reporter1)
simulation.reporters.append(reporter2)
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True))
simulation.reporters.append(StateDataReporter("md_log.txt", 100, step=True,
        potentialEnergy=True, temperature=True, volume=True))

print("Running NVT")
simulation.step(2000)

Running NVT
#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)"
1000,-11362.734104145115,290.55084148073956,123.37574384135401
2000,-11345.606998432224,307.0269565866364,123.37574384135401


In [10]:
reporter1._traj_file.n_atoms

1960

In [11]:
reporter1._traj_file.n_frames

2

In [12]:
reporter2._traj_file.n_frames

200

In [13]:
reporter2._traj_file._handle.createVariable

<bound method Dataset.createVariable of <class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    title: CREATED at 2024-05-06 17:35:25.515348 on jessica-lenovo
    application: Omnia
    program: MDTraj
    programVersion: 1.9.10.dev0
    Conventions: AMBER
    ConventionVersion: 1.0
    dimensions(sizes): frame(200), spatial(3), atom(1960), cell_spatial(3), cell_angular(3), label(5)
    variables(dimensions): float64 cell_lengths(frame, cell_spatial), float64 cell_angles(frame, cell_angular), |S1 cell_spatial(cell_spatial), |S1 cell_angular(cell_spatial, label), float32 time(frame), float32 coordinates(frame, atom, spatial), |S1 spatial(spatial)
    groups: >

In [14]:
# clear reporters
simulation.reporters.clear()

## Patch to exclude `netCDF4` for behavior test

This emulates an environment where netCDF4 is not installed and the SciPy fallback is used.

In [15]:
import sys

In [16]:
# unload netCDF4 and mdtraj

sys.modules["netCDF4"] = None

if 'mdtraj.reporters.NetCDFReporter' in sys.modules:
    del sys.modules['mdtraj.reporters.NetCDFReporter']

if 'mdtraj.reporters' in sys.modules:
    del sys.modules['mdtraj.reporters']

if 'mdtraj' in sys.modules:
    del sys.modules['mdtraj']

In [17]:
from mdtraj.reporters import NetCDFReporter

In [18]:
reporter = NetCDFReporter('trajectory_netcdf4.nc', 1000)

For improved performance, consider installing the 'netCDF4' module. See installation instructions at:
https://unidata.github.io/netcdf4-python/#quick-install


In [19]:
print(type(reporter._traj_file._handle))

<class 'scipy.io._netcdf.netcdf_file'>


In [20]:
simulation.reporters.append(reporter)
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True))
simulation.reporters.append(StateDataReporter("md_log.txt", 100, step=True,
        potentialEnergy=True, temperature=True, volume=True))

In [21]:
print("Running NVT")
simulation.step(2000)

Running NVT
#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)"
3000,-11383.914951313083,289.8766910514546,123.37574384135401
4000,-11462.98418043906,301.658394906099,123.37574384135401


In [22]:
reporter._traj_file.n_atoms

1960

In [23]:
reporter._traj_file.n_frames

2